# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
from pprint import pprint
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data_df=pd.read_csv("../output_data/cities.csv")
city_data_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [7]:
locations=city_data_df[["Lat","Lng"]]
humidity=city_data_df[["Humidity"]]
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations, weights = humidity,dissipating=False,max_intensity=300, point_radius=5)
fig.add_layer(heat_layer)

TraitError: Humidity is not a valid weight. Weights must be floats

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
ideal_weather_df=city_data_df.loc[
    (city_data_df["Max Temp"]>70)&(city_data_df["Max Temp"]<80)& \
    (city_data_df["Wind Speed"]<10)& \
    (city_data_df["Cloudiness"]==0)
].dropna()
ideal_weather_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df=ideal_weather_df[["City","Country","Lat","Lng"]]
hotel_df["Hotel Name"]=""
hotel_df

<ipython-input-14-b96651bcee6d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""


,City,Country,Lat,Lng,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,
88,cidreira,BR,-30.17,-50.22,
138,morondava,MG,-20.30,44.28,
139,vaini,IN,15.34,74.49,
176,nador,MA,35.17,-2.93,
297,mogok,MM,22.92,96.51,
444,birjand,IR,32.86,59.22,


In [22]:
params={
    "radius" :5000,
    "types":"lodging",
    "key":g_key
}
for index, row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    
    params["location"]=f"{lat},{lng}"
    base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    name_address=requests.get(base_url,params=params).json()

    try:
        hotel_df.loc[index,"Hotel Name"]=name_address["results"][0]["name"]
    except:
        pass
    
hotel_df

/Users/valentinazhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,City,Country,Lat,Lng,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,Hotel Escarpas do Lago
88,cidreira,BR,-30.17,-50.22,Hotel Castelo
138,morondava,MG,-20.30,44.28,La Case Bambou
139,vaini,IN,15.34,74.49,Dandeli Lake County
176,nador,MA,35.17,-2.93,Hôtel La Giralda
297,mogok,MM,22.92,96.51,MOGOK HILL HOTEL
444,birjand,IR,32.86,59.22,Birjand Tourist Hotel


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map

markeer_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
# Display figure
